In [ ]:
%env MUJOCO_GL=egl

import os
import mujoco
import mediapy
import numpy as np
import pinocchio as pin
from scipy.spatial.transform import Rotation

from discoverse import DISCOVERSE_ASSERT_DIR

np.set_printoptions(precision=5, suppress=True, linewidth=500)

In [ ]:
def get_site_tmat(mj_data, site_name):
    tmat = np.eye(4)
    tmat[:3,:3] = mj_data.site(site_name).xmat.reshape((3,3))
    tmat[:3,3] = mj_data.site(site_name).xpos
    return tmat

def get_camera_tmat(mj_data, cam_name):
    tmat = np.eye(4)
    tmat[:3,:3] = mj_data.cam(cam_name).xmat.reshape(3,3)
    tmat[:3,3] = mj_data.cam(cam_name).xpos
    return tmat

def get_body_tmat(mj_data, body_name):
    tmat = np.eye(4)
    tmat[:3,:3] = Rotation.from_quat(mj_data.body(body_name).xquat[[1,2,3,0]]).as_matrix()
    tmat[:3,3] = mj_data.body(body_name).xpos
    return tmat

In [ ]:
mjcf_file_path = "mjcf/exhibition_conference.xml"
mjcf_file = os.path.join(DISCOVERSE_ASSERT_DIR, mjcf_file_path)

mj_model = mujoco.MjModel.from_xml_path(mjcf_file)
mj_data = mujoco.MjData(mj_model)

renderer = mujoco.Renderer(mj_model, 480, 640)

options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
options.flags[mujoco.mjtVisFlag.mjVIS_COM] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_PERTFORCE] = True
# options.flags[mujoco.mjtVisFlag.mjVIS_PERTOBJ] = True
# options.frame = mujoco.mjtFrame.mjFRAME_BODY.value
options.frame = mujoco.mjtFrame.mjFRAME_SITE.value

mj_data.qpos[:] = mj_model.key("carry").qpos

mujoco.mj_forward(mj_model, mj_data)

renderer.update_scene(mj_data, -1, options)
rgb = renderer.render()
mediapy.show_image(rgb)